# Splunk App for Data Science and Deep Learning - Neural Network Designer

## Neural Network for Binary Classification with Dense Layers and Embeddings

This notebook contains an example workflow how to work on custom containerized code that seamlessly interfaces with the Splunk App for Data Science and Deep Learning. As an example we use a custom binary neural network classifier built on keras and tensorflow. We refer to the TensorFlow tutorial for structured data with the feature columns feature: <a href="https://www.tensorflow.org/tutorials/structured_data/feature_columns">https://www.tensorflow.org/tutorials/structured_data/feature_columns</a>

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# mltkc_import
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import shap

# restrict GPU memory https://www.tensorflow.org/guide/gpu#limiting_gpu_memory_growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            # print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

def df_to_dataset(dataframe, target_label_name, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop(target_label_name)
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

def df_to_dataset_apply(dataframe, batch_size=32):
    dataframe = dataframe.copy()
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe)))
    ds = ds.batch(batch_size)
    return ds

/opt/conda/lib/python3.8/site-packages/tensorflow/python/ops/random_ops.py:285: SyntaxWarning: "is" with a literal. Did you mean "=="?
  minval_is_zero = minval is 0  # pylint: disable=literal-comparison
/opt/conda/lib/python3.8/site-packages/tensorflow/python/ops/random_ops.py:286: SyntaxWarning: "is" with a literal. Did you mean "=="?
  maxval_is_one = maxval is 1  # pylint: disable=literal-comparison
/opt/conda/lib/python3.8/site-packages/tensorflow/python/ops/ragged/ragged_batch_gather_with_default_op.py:84: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if (default_value.shape.ndims is not 0
/opt/conda/lib/python3.8/site-packages/tensorflow/python/ops/ragged/ragged_batch_gather_with_default_op.py:85: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  and default_value.shape.ndims is not 1):
"is not" with a literal. Did you mean "!="?
"is" with a literal. Did you mean "=="?
"is" with a literal. Did you mean "=="?
"is not" with a literal. Did you mean "!="?


In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("TensorFlow version: " + tf.__version__)
print("Keras version: " + keras.__version__)
print("SHAP version: " + shap.__version__)
if gpus:
    for gpu in gpus:
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

numpy version: 1.19.2
pandas version: 1.1.3
TensorFlow version: 2.2.0
Keras version: 2.3.0-tf
SHAP version: 0.37.0


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a prepared dataset into this environment.

| inputlookup diabetic.csv <br>
| eval _time = now() - random()%(3600*24) <br>
| table _time * <br>
| eval diabetesMed01 = if(diabetesMed=="Yes",1.0,0.0) <br>
| fit MLTKContainer mode=stage algo=binary_nn_classifier_designer epochs=10 batch_size=4 structure=32-16 numeric_features="admission_source_id admission_type_id" categorical_features="acetohexamide acarbose age" embedding_features="" diabetesMed01 from admission_source_id admission_type_id  acetohexamide acarbose age  into app:diabetes_med_model<br>

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("my_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [9]:
# mltkc_stage
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [10]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
df, param = stage("diabetes_med_model")
print(df.shape)
print(str(param))
df

(10000, 6)
{'options': {'params': {'mode': 'stage', 'algo': 'binary_nn_classifier_designer', 'epochs': '10', 'batch_size': '4', 'structure': '32-16', 'numeric_features': '"admission_source_id admission_type_id"', 'categorical_features': '"acetohexamide acarbose age"', 'embedding_features': '""'}, 'args': ['diabetesMed01', 'admission_source_id', 'admission_type_id', 'acetohexamide', 'acarbose', 'age'], 'target_variable': ['diabetesMed01'], 'feature_variables': ['admission_source_id', 'admission_type_id', 'acetohexamide', 'acarbose', 'age'], 'model_name': 'diabetes_med_model', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'handle_new_cat': 'default', 'max_distinct_cat_values': '100', 'max_distinct_cat_values_for_classifiers': '100', 'max_distinct_cat_values_for_scoring': '100', 'max_fit_time': '600', 'max_inputs': '100000', 'max_memory_usage_mb': '4000', 'max_model_size_mb': '30', 'max_score_time': '600', 'use_sampling': 'true'}, 'kfold_cv': None}, 'feature_variables': ['admission_sourc

,age,admission_type_id,admission_source_id,acetohexamide,acarbose,diabetesMed01
0,[0-10),6,1,No,No,0.0
1,[10-20),1,7,No,No,1.0
2,[20-30),1,7,No,No,1.0
3,[30-40),1,7,No,No,1.0
4,[40-50),1,7,No,No,1.0
...,...,...,...,...,...,...
9995,[80-90),5,17,No,No,1.0
9996,[80-90),2,1,No,No,0.0
9997,[50-60),1,7,No,No,0.0
9998,[50-60),1,7,No,No,0.0


## Stage 2 - create and initialize a model

In [12]:
# mltkc_init
# initialize the model
# params: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    model['param'] = param
    #y = df[param['target_variables'][0]]
    #X = df[param['feature_variables']] #.astype(float)
    #print("FIT build model with input shape " + str(X.shape))
    #input_shape = int(X.shape[1])
    
    model_structure = '256-128'
    numeric_features = None
    embedding_features = None
    embedding_dimensions = 8
    categorical_features = None

    feature_columns = []
    
    if 'options' in param:
        if 'params' in param['options']:
            if 'structure' in param['options']['params']:
                model_structure = str(param['options']['params']['structure']).lstrip("\"").rstrip("\"").lstrip(" ").rstrip(" ")
            if 'numeric_features' in param['options']['params']:
                numeric_features = str(param['options']['params']['numeric_features']).lstrip("\"").rstrip("\"").lstrip(" ").rstrip(" ").replace(" ", ",").split(",")
                for feature in numeric_features:
                    if '*' in feature:
                        wildcards = df.filter(like=feature.replace('*','')).columns
                        for wildcard in wildcards:
                            feature_columns.append(feature_column.numeric_column(wildcard))
                    elif feature in df:
                        feature_columns.append(feature_column.numeric_column(feature))
            if 'embedding_features' in param['options']['params']:
                embedding_features = str(param['options']['params']['embedding_features']).lstrip("\"").rstrip("\"").lstrip(" ").rstrip(" ").replace(" ", ",").split(",")
                for feature in embedding_features:
                    if '*' in feature:
                        wildcards = df.filter(like=feature.replace('*','')).columns
                        for wildcard in wildcards:
                            feature_embedding = feature_column.categorical_column_with_vocabulary_list(wildcard, df[wildcard].unique())
                            feature_embedding = feature_column.embedding_column(feature_embedding, dimension=embedding_dimensions)
                            feature_columns.append(feature_embedding)
                    elif feature in df:
                        feature_embedding = feature_column.categorical_column_with_vocabulary_list(feature, df[feature].unique())
                        feature_embedding = feature_column.embedding_column(feature_embedding, dimension=embedding_dimensions)
                        feature_columns.append(feature_embedding)
            if 'categorical_features' in param['options']['params']:
                categorical_features = str(param['options']['params']['categorical_features']).lstrip("\"").rstrip("\"").lstrip(" ").rstrip(" ").replace(" ", ",").split(",")
                for feature in categorical_features:
                    if '*' in feature:
                        wildcards = df.filter(like=feature.replace('*','')).columns
                        for wildcard in wildcards:
                            categorical_column = feature_column.categorical_column_with_vocabulary_list(wildcard, df[wildcard].unique())
                            categorical_column = feature_column.indicator_column(categorical_column)
                            feature_columns.append(categorical_column)
                    elif feature in df:
                        categorical_column = feature_column.categorical_column_with_vocabulary_list(feature, df[feature].unique())
                        categorical_column = feature_column.indicator_column(categorical_column)
                        feature_columns.append(categorical_column)
                    
    model['feature_columns'] = feature_columns
    feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
    model['feature_layer'] = feature_layer
    
    hidden_factors = np.floor(np.array(model_structure.split("-"), dtype="float"))
    keras_model = tf.keras.Sequential()
    keras_model.add(feature_layer)
    for hidden in hidden_factors:
        keras_model.add(layers.Dense(int(hidden), activation=tf.nn.relu))
        keras_model.add(layers.Dropout(0.01))
    keras_model.add(layers.Dense(1, activation=tf.nn.sigmoid))
        
    keras_model.compile(optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy'])
    model['keras_model'] = keras_model
        
    return model

In [13]:
# test mltkc_stage_create_model
model = init(df,param)
#print(model.summary())

In [14]:
print(model['feature_columns'])

[NumericColumn(key='admission_source_id', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='admission_type_id', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='acetohexamide', vocabulary_list=('No',), dtype=tf.string, default_value=-1, num_oov_buckets=0)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='acarbose', vocabulary_list=('No', 'Steady', 'Up'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='age', vocabulary_list=('[0-10)', '[10-20)', '[20-30)', '[30-40)', '[40-50)', '[50-60)', '[60-70)', '[70-80)', '[80-90)', '[90-100)'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]


## Stage 3 - fit the model

In [15]:
# mltkc_stage_create_model_fit
# returns a fit info json object
def fit(model,df,param):
    returns = {}
    #X = df[param['feature_variables']]
    #Y = df[param['target_variables']]
    train, test = train_test_split(df, test_size=0.2)
    train, val = train_test_split(train, test_size=0.2)
    print(len(train), 'train examples')
    print(len(val), 'validation examples')
    print(len(test), 'test examples')
        
    model_epochs = 10
    model_batch_size = 1
    if 'options' in param:
        if 'params' in param['options']:
            if 'epochs' in param['options']['params']:
                model_epochs = int(param['options']['params']['epochs'])
            if 'batch_size' in param['options']['params']:
                model_batch_size = int(param['options']['params']['batch_size'])
    
    train_ds = df_to_dataset(df, param['target_variables'][0], batch_size=model_batch_size)
    val_ds = df_to_dataset(val, param['target_variables'][0], shuffle=False, batch_size=model_batch_size)
    test_ds = df_to_dataset(test, param['target_variables'][0], shuffle=False, batch_size=model_batch_size)

    # connect model training to tensorboard
    log_dir="/srv/notebooks/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    
    # run the training
    returns['fit_history'] = model['keras_model'].fit(train_ds,
        validation_data=val_ds,
        epochs=model_epochs,
        verbose=2,
        callbacks=[tensorboard_callback])    

    returns['model_epochs'] = model_epochs
    returns['model_batch_size'] = model_batch_size
    model['model_epochs'] = model_epochs
    model['model_batch_size'] = model_batch_size
    
    returns['model_loss_acc'] = model['keras_model'].evaluate(test_ds)
    return returns


In [16]:
returns = fit(model,df,param)
print(returns['model_loss_acc'])

6400 train examples
1600 validation examples
2000 test examples
Epoch 1/10
2500/2500 - 8s - loss: 0.5798 - accuracy: 0.7408 - val_loss: 0.5675 - val_accuracy: 0.7437
Epoch 2/10
2500/2500 - 7s - loss: 0.5713 - accuracy: 0.7410 - val_loss: 0.5641 - val_accuracy: 0.7437
Epoch 3/10
2500/2500 - 10s - loss: 0.5666 - accuracy: 0.7417 - val_loss: 0.5644 - val_accuracy: 0.7437
Epoch 4/10
2500/2500 - 8s - loss: 0.5653 - accuracy: 0.7417 - val_loss: 0.5745 - val_accuracy: 0.7444
Epoch 5/10
2500/2500 - 10s - loss: 0.5659 - accuracy: 0.7430 - val_loss: 0.5588 - val_accuracy: 0.7450
Epoch 6/10
2500/2500 - 9s - loss: 0.5639 - accuracy: 0.7418 - val_loss: 0.5587 - val_accuracy: 0.7456
Epoch 7/10
2500/2500 - 7s - loss: 0.5635 - accuracy: 0.7424 - val_loss: 0.5601 - val_accuracy: 0.7456
Epoch 8/10
2500/2500 - 7s - loss: 0.5631 - accuracy: 0.7423 - val_loss: 0.5578 - val_accuracy: 0.7456
Epoch 9/10
2500/2500 - 11s - loss: 0.5619 - accuracy: 0.7427 - val_loss: 0.5568 - val_accuracy: 0.7456
Epoch 10/10
250

## Stage 4 - apply the model

In [17]:
# mltkc_stage_create_model_apply
def apply(model,df,param):
    X = df[param['feature_variables']]
    model_batch_size = 1
    print("APPLY PARAMS: " + str(param))
    if 'options' in param:
        if 'params' in param['options']:
            if 'batch_size' in param['options']['params']:
                model_batch_size = int(param['options']['params']['batch_size'])
    # TODO
    apply_dataset = df_to_dataset_apply(X, batch_size=model_batch_size)
    y_hat = model['keras_model'].predict(apply_dataset, verbose=1)
    return y_hat

In [18]:
# test mltkc_stage_create_model_apply
y_hat = apply(model,df,param)
print(y_hat)

APPLY PARAMS: {'options': {'params': {'mode': 'stage', 'algo': 'binary_nn_classifier_designer', 'epochs': '10', 'batch_size': '4', 'structure': '32-16', 'numeric_features': '"admission_source_id admission_type_id"', 'categorical_features': '"acetohexamide acarbose age"', 'embedding_features': '""'}, 'args': ['diabetesMed01', 'admission_source_id', 'admission_type_id', 'acetohexamide', 'acarbose', 'age'], 'target_variable': ['diabetesMed01'], 'feature_variables': ['admission_source_id', 'admission_type_id', 'acetohexamide', 'acarbose', 'age'], 'model_name': 'diabetes_med_model', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'handle_new_cat': 'default', 'max_distinct_cat_values': '100', 'max_distinct_cat_values_for_classifiers': '100', 'max_distinct_cat_values_for_scoring': '100', 'max_fit_time': '600', 'max_inputs': '100000', 'max_memory_usage_mb': '4000', 'max_model_size_mb': '30', 'max_score_time': '600', 'use_sampling': 'true'}, 'kfold_cv': None}, 'feature_variables': ['admission_so

## Stage 5 - save the model

In [19]:
# save model to name in expected convention "<algo_name>_<model_name>.h5"
def save(model,name):
    # save keras model to hdf5 file
    # https://www.tensorflow.org/beta/tutorials/keras/save_and_restore_models
    if 'keras_model' in model:
        tf.keras.models.save_model(model['keras_model'], MODEL_DIRECTORY + name)
    return model

In [20]:
save(model,'diabetes_med_model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /srv/app/model/data/diabetes_med_model/assets


{'param': {'options': {'params': {'mode': 'stage',
    'algo': 'binary_nn_classifier_designer',
    'epochs': '10',
    'batch_size': '4',
    'structure': '32-16',
    'numeric_features': '"admission_source_id admission_type_id"',
    'categorical_features': '"acetohexamide acarbose age"',
    'embedding_features': '""'},
   'args': ['diabetesMed01',
    'admission_source_id',
    'admission_type_id',
    'acetohexamide',
    'acarbose',
    'age'],
   'target_variable': ['diabetesMed01'],
   'feature_variables': ['admission_source_id',
    'admission_type_id',
    'acetohexamide',
    'acarbose',
    'age'],
   'model_name': 'diabetes_med_model',
   'algo_name': 'MLTKContainer',
   'mlspl_limits': {'handle_new_cat': 'default',
    'max_distinct_cat_values': '100',
    'max_distinct_cat_values_for_classifiers': '100',
    'max_distinct_cat_values_for_scoring': '100',
    'max_fit_time': '600',
    'max_inputs': '100000',
    'max_memory_usage_mb': '4000',
    'max_model_size_mb': '30'

## Stage 6 - load the model

In [21]:
# load model from name in expected convention "<algo_name>_<model_name>.h5"
def load(name):
    model = {}
    #with open(MODEL_DIRECTORY + name + ".feature_layer_config.json", 'r') as file:
    #    feature_layer_config = json.load(file)

    # #model = tf.keras.models.load_model(MODEL_DIRECTORY + name + '.h5') #, custom_objects=feature_layer_config)
    model['keras_model'] = tf.keras.models.load_model(MODEL_DIRECTORY + name)
    
    return model

In [23]:
model2 = load('diabetes_med_model')

In [24]:
y_hat = apply(model2,df,param)
print(y_hat)

APPLY PARAMS: {'options': {'params': {'mode': 'stage', 'algo': 'binary_nn_classifier_designer', 'epochs': '10', 'batch_size': '4', 'structure': '32-16', 'numeric_features': '"admission_source_id admission_type_id"', 'categorical_features': '"acetohexamide acarbose age"', 'embedding_features': '""'}, 'args': ['diabetesMed01', 'admission_source_id', 'admission_type_id', 'acetohexamide', 'acarbose', 'age'], 'target_variable': ['diabetesMed01'], 'feature_variables': ['admission_source_id', 'admission_type_id', 'acetohexamide', 'acarbose', 'age'], 'model_name': 'diabetes_med_model', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'handle_new_cat': 'default', 'max_distinct_cat_values': '100', 'max_distinct_cat_values_for_classifiers': '100', 'max_distinct_cat_values_for_scoring': '100', 'max_fit_time': '600', 'max_inputs': '100000', 'max_memory_usage_mb': '4000', 'max_model_size_mb': '30', 'max_score_time': '600', 'use_sampling': 'true'}, 'kfold_cv': None}, 'feature_variables': ['admission_so

## Stage 7 - provide a summary of the model

In [25]:
# return model summary
def summary(model=None):
    returns = {"version": {"tensorflow": tf.__version__, "keras": keras.__version__} }
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code

In [26]:
summary(model2)

{'version': {'tensorflow': '2.2.0', 'keras': '2.3.0-tf'}}